
# Библиотека Prophet для прогнозирования временных рядов


## Machine Learning

Или __«машинное обучение»__ — множество методов для разработки алгоритмов, которые решают задачу на основании поиска закономерностей в данных.  
  
В компаниях очень много данных, и логично, что бизнес хочет их использовать. 
  
__Наиболее частая задача — прогнозирование различных показателей:__
- количество пользователей
- выручка
- вероятность того, что пользователь совершит какое-то действие (например, сделает покупку или, наоборот, перестанет пользоваться продуктом)
- и т. д.
  
Очень популярен __анализ временных рядов__. Временные ряды — это последовательность значений в определенный момент времени или за определенный промежуток времени. __Например:__
- количество покупок по дням
- выручка по дням/неделям/месяцам
- и т. д.

# Задача

Сеть из трех продуктовых магазинов хочет спрогнозировать общий объем выручки сети на январь 2021 года. В качестве данных для построения моделей компания предоставляет данные о выручке за период `2020.05.01 - 2020.12.31`. Прогноз должен быть по дням.

### Как это сделать?

1. Исследуем данные, которые нам предоставили
2. Построим стратегию расчета прогнозной модели
3. Сделаем машин лернинг

### Подготовим нужные библиотеки

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import warnings # это библиотека для управления warning'ами — то есть различными предупреждениями
                # ниже мы с помощью нее игнорим лишние предупреждения
                # мы можем это убрать, но где-то всплывали ворнинги, и чтобы они не раздражали и не вызывали вопросов, мы их отключим

warnings.filterwarnings('ignore')

### Графический анализ данных

Для начала посмотрим на данные — какие есть поля, объем данных и т. д.

In [ ]:
!wget -O revenue_01.05.2020_31.12.2020.csv https://res.cloudinary.com/djcwxgbfz/raw/upload/v1612457412/skills/revenue_01.05.2020_31.12.2020.csv

data_revenue = pd.read_csv('revenue_01.05.2020_31.12.2020.csv') # подключимся к данным

In [ ]:
data_revenue.head()

In [ ]:
data_revenue.shape 

В наших данных есть общая информация о выручке в каждом из трех филиалов за каждый день с 1 мая 2020 по 31 декабря 2020.

Наша задача — спрогнозировать общую выручку по всем трем филиалам, поэтому посмотрим на общие данные.

Возьмем гистограмму, которая отдаст частотность разных показателей выручки. В сам график передадим сгруппированный датасет по датам и выручке, чтобы каждый день встречался единожды и мы считали сумму по трем филиалам.

In [ ]:
# суммарная выручка по всем филиалам

plt.figure(figsize=(10, 5)) 
plt.title('Распределение общей выручки') 
plt.xlabel('Выручка') 
plt.ylabel('Количество дней') 
plt.grid() 

plt.hist(data_revenue.groupby('date')['revenue'].sum(), bins=50) 

plt.show()

Видим, что в общей выручке явно есть смесь. Попробуем построить гистограмму выручки без группирования по дню. 

Теперь за каждый день у нас будет 3 показателя за 1,2 и 3 филиал.

In [ ]:
plt.figure(figsize=(10, 5)) 
plt.title('Распределение общей выручки') 
plt.xlabel('Выручка') 
plt.ylabel('Количество дней') 
plt.grid() 

plt.hist(data_revenue['revenue'], bins=150) # можем увеличить втрое количество бинов, чтобы визуализация была точнее

plt.show()

Очевидная смесь в данных. Посмотрим, решается ли это разделением на филиалы. Для этого построим гистограммы отдельно по филиалам 

с помощью метода `.unique` возьмем список уникальных филиалов и пройдем по нему циклом:

In [ ]:
plt.figure(figsize=(10, 5)) 

filial = pd.unique(data_revenue['filial']) # сделаем серию со списком филиалов

for f in filial: # фильтр по филиалу
    sns.distplot(data_revenue.loc[data_revenue['filial'] == f]['revenue'], label = f);
    
plt.legend();

Как видим, во многом смесь объясняет разделение на филиалы, но не до конца.  
Посмотрим, как выглядят данные в виде временных рядов (в сумме и в разделении по филиалам)

### Отрисуем временной ряд

Используем объект `dates` из библиотеки `matplotlib`, чтобы корректно отображать дату. 

В настройках графика мы укажем с помощью метода `.MonthLocator()`, что хотим отображать даты по месяцам, а не по дням. 

- запишем его в переменную `locator` месячную настройку подписей

- с помощью метода `.gca()` сообщим о намерении инициировать создание оси (`axis`), в нашем случае ось Х

- применим к оси метод `set_major_locator()` с месячной настройкой, которая лежит в переменной 


In [ ]:
import matplotlib.dates as mdates #импортируем для локаторов

# суммарная выручка
plt.figure(figsize = (15, 7))

plt.title('Общая динамика выручки') 
plt.xlabel('Дата') 
plt.ylabel('Выручка') 
plt.grid()
locator = mdates.MonthLocator() # передадим, что хотим подписи помесячно
X = plt.gca().xaxis # инициируем ось х
X.set_major_locator(locator) # передадим в ось параметры локатора
sns.lineplot(x = data_revenue.groupby('date')['revenue'].sum().index, # .index, чтобы взять дату по оси x
             y = data_revenue.groupby('date')['revenue'].sum()
            )
plt.show()

И теперь проделаем то же самое для всех филиалов, используя цикл.

In [ ]:
# выручка по филиалам
plt.figure(figsize = (15, 7))

plt.title('Динамика выручки по филиалам') 
plt.xlabel('Дата') 
plt.ylabel('Выручка') 
plt.grid()
for f in pd.unique(data_revenue['filial']):
    sns.lineplot( 
                 x = data_revenue.loc[data_revenue['filial'] == f]['date'], 
                 y = data_revenue.loc[data_revenue['filial'] == f]['revenue'], 
                 label = f);
    
locator = mdates.MonthLocator()
X = plt.gca().xaxis
X.set_major_locator(locator)

На основании графического анализа данных можно сделать следующие выводы и предположения:
- в данных есть смесь, вызванная тем, что разные филиалы в среднем имеют разную выручку
- в данных можно увидеть тренды (возрастающий тренд для филиалов 1 и 2 и убывающий тренд для филиала 3)
- в данных есть смесь, вызванная скорее сезонными явлениями (на это указывает циклический характер временных рядов)

Таким образом в качестве потенциальных гипотез, которые помогут улучшить качество прогнозной модели, стоит учесть:
- разделение по филиалам
- выделение линейного тренда
- добавление учета сезонности

# Построение прогнозной модели

## Prophet 

Prophet — очень популярная библиотека от компании Facebook для анализа и прогнозирования временных рядов. 

- просто настраиваются прогнозные модели
- можно полностью довериться алгоритму и использовать его как «черный ящик»
- можно очень детально настроить множество параметров, которые доступны в алгоритме  

__Модель__ — это алгоритм или набор алгоритмов, которые строятся на основании данных и с помощью которого/которых мы сможем прогнозировать.

__Посмотрим [документацию](https://facebook.github.io/prophet/docs/quick_start.html)__


Для построения модели нам нужно привести данные к стандартному виду, и нам понадобятся данные обучающей выборки.

## Обучающая/контрольная выборки 

В машинном обучении есть понятия «обучение с учителем» и «обучение без учителя». То есть имеются ли у нас ответы и подсказки, чтобы проверить, хорошо ли обучилась модель.

Мы можем обучить нашу модель на данных с мая по ноябрь, а на декабрьских проверить, хорошо ли наша модель предсказывает. Разделим данные на обучающую и контрольную группы. 


<div style="display: flex; width: 100%; font-family: Arial">
<div style="width: 100%; margin-left: 0px">
  <div style="font-size: 20px; line-height: 30px">Обучающие данные</div>

  <div
    style="
      font-size: 14px;
      line-height: 20px;
      color: rgba(0, 0, 0, 0.48);
      margin-top: 16px;
    "
  >
    Основа для обучения модели. 
      <br/>
      Они выступают неким эталоном динамики и поведения данных.  
  </div>
</div>

<div style="width: 100%; margin-left: 24px">
  <div style="font-size: 20px; line-height: 30px">Контрольные данные</div>

  <div
    style="
      font-size: 14px;
      line-height: 20px;
      color: rgba(0, 0, 0, 0.48);
      margin-top: 16px;
    "
  >
    Нужны для проверки, как хорошо обучилась модель и на сколько точно она может предсказывать будущее. 
  </div>
</div>

<div style="width: 100%; margin-left: 24px">
  <div style="font-size: 20px; line-height: 30px">Переобучение модели</div>

  <div
    style="
      font-size: 14px;
      line-height: 20px;
      color: rgba(0, 0, 0, 0.48);
      margin-top: 16px;
    "
  >
    Ситуация, когда модель дает качество в обучении, но низкое на контроле, взяв единичные особенности обучающей выборки как общее поведение данных.
   
  </div>
</div>
</div>


<img style='float:left' src="https://raw.githubusercontent.com/foobar167/articles/master/Machine_Learning/Brochure/data/Ris2.6-Nedoobucheniye-optimum-i-pereobucheniye-v-regressii.png" width="700"/>

В нашем случае, как и договорились, разделим данные на обучение и контроль следующим образом:
- данные с мая по ноябрь — обучение
- данные за декабрь — контроль

In [ ]:
train = data_revenue.loc[data_revenue['date'] < '2020-12-01']
test = data_revenue.loc[data_revenue['date'] >= '2020-12-01']

train.shape, test.shape

### Попробуем построить первую модель для общей выручки

Для начала сгруппируем данные каждой выборки по дате с суммой выручки. Индекс сбросим, чтобы работать только с колонками.


In [ ]:
train_0 = train.groupby(['date'])['revenue'].sum().reset_index() # сгруппировали данные обучающей выборки
test_0 = test.groupby(['date'])['revenue'].sum().reset_index() # сгруппировали данные контрольной выборки

In [ ]:
train_0.head()

Установим и импортируем `Prophet()`. Несколько дополнительных способой установки можно найти в отдельном ноутбуке. 

In [ ]:
!pip install pystan
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

Сначала попробуем построить прогнозную модель без указания каких бы то ни было параметров — пусть алгоритм действует на свое усмотрение.

__Параметры__ — это настройки модели. Обученная модель фиксирует ряд настроек, и мы на этапе обучения можем подсказать ей, как настраиваться или оставить настройки  «по умолчанию» . 

Чтобы создать модель, мы используем 
1. объект `Prophet()` из библиотеки

2. метод `.fit()` для подгонки модели
3. правильно подготовленные данные


Создадим `model`, присвоив ей объект `Prophet()`

In [ ]:
model = Prophet()
model

`.fit()` — метод подгонки модели под заданные данные и параметры модели

Нужно, чтобы в метод `.fit()` подавался датафрейм с нужными полями. Заглянем еще раз в [документацию.](https://facebook.github.io/prophet/docs/quick_start.html)
- `ds` — временная гранула, в нашем случае день

- `y` — значения, которые будем предсказывать, в нашем случае выручка

In [ ]:
train_0.head()

Переименуем столбцы в обучающем и тестовом датасетах, чтобы они подходили для использования методов Prophet

In [ ]:
train_0.columns = ['ds', 'y'] # переименовали столбцы

test_0.columns = ['ds', 'y']

In [ ]:
model.fit(train_0) # подогнали модель под наши данные

Как видим, алгоритм сам нам подсказывает, что он проигнорировал, выбирая параметры  

1. `INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.`

    - годовую сезонность (*для годовой сезонности нам нужно иметь данные минимум за два года, чтобы суметь использовать ее в модели*)


2. `INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.`

    - дневную сезонность (*дневная сезонность может использоваться в случае, если данные собираются по часам/минутам, в нашем случае данные представлены по дням*).


Зато он обнаружил недельную сезонность и использовал его при настройке модели.


## Качество модели

Посмотрим на качество получившейся модели. Для этого нам необходимо задать горизонт прогнозирования (в нашем случае это месяц) и создать дата-фрейм с датами из «будущего», для которого модель потом будет строить прогноз.

`make_future_dataframe()` — метод профета, который создает дата-фрейм с временным периодом будущего. В аргумент ему мы передаем `periods` и задаем количество элементов, на которые хотим получить прогноз. 

Так как наши данные имеют дневную гранулярность, прогноз мы также получим по дням от последней даты на количество дней, указанное в скобках.

In [ ]:
future = model.make_future_dataframe(periods=31) # говорим профету сделать дата-фрейм на 31 день
future.tail(31) # выводим 31 строку с конца

Теперь можем построить прогноз методом __`.predict`__.

Применим его к нашей модели и запишем в отдельную переменную.

доверительный интервал по умолчанию 95%, это популярный стандарт, который вполне нас устраивает

In [ ]:
forecast = model.predict(future)
forecast.head() # возвращает много колонок

Основные поля в прогнозе следующие:
- `ds` — дата прогноза

- `yhat` — спрогнозированное значение

- `yhat_lower` — нижняя граница доверительного интервала для прогноза
- `yhat_upper` — верхняя граница доверительного интервала для прогноза


In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail() # оставим только нужные

Также с помощью метода `.plot()` прогноз можно построить на графике и посмотреть визуально его адекватность.

Черные точки — наши данные.

In [ ]:
model.plot(forecast);

Кроме того, Prophet позволяет также наглядно разложить ряд на основные компоненты — тренд и сезонность:

`plot_components()` — возвращает несколько графиков, среди которых тренд и столько сезонностей, сколько он найдет.

Точка с запятой в конце строки позволяет не выводить лишнего. Обычно такая проблема возникает с графиками — выводится лишняя информация или даже дублирующиеся графики.

`Suppress output
Put a ‘;’ at the end of a line to suppress the printing of output. This is useful when doing calculations which generate long output you are not interested in seeing. It also keeps the object out of the output cache, so if you’re working with large temporary objects, they’ll be released from memory sooner.`

In [ ]:
model.plot_components(forecast);

Видим, что тренд у общей выручки убывающий и у нас есть четкая недельная сезонность:
- наибольшее количество покупок люди совершают в субботу
- чуть меньше в воскресенье
- среди будних дней больше всего покупок в понедельник и вторник

## Ошибка прогноза

Давайте посмотрим, какая получилась ошибка прогноза. По модели мы строим прогноз на тестовый период, затем сравниваем его с контрольными данными и считаем отклонение.

### Как посчитать отклонение?

Ошибку прогноза мы посчитаем с помощью функции `mean absolute error()` (среднее значение модулей отклонений прогноза от факта) из библиотеки `scikit-learn`:

В аргументы он принимает серию предсказанных значений и серию с контрольными значениями.  

Что под капотом у функции: из факта вычитаем прогноз, берем от этого модуль (положительное значение) и считаем среднее значение по всем модулям. 


In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(forecast['yhat'].tail(31), test_0['y']) 

Сходу не понять, сильное ли отклонение мы получили. Посмотрим в %.

Для того чтобы посчитать `отклонение в процентах`, необходимо `модули отклонений прогноза от факта` разделить на `факт`, от полученных значений посчитать среднее арифметическое и умножить на 100.

`np.mean()` — функция библиотеки NumPy считает среднее арифметическое

`np.abs()` — функция библиотеки NumPy, которая вернет модули

In [ ]:
modul_dif = np.abs(forecast.tail(31).reset_index()['yhat'] - test_0['y']) # положим модули отклонений от прогноза в отдельную переменную

dif = np.mean(modul_dif/test_0['y']) # считаем среднее арифметическое отклонение 

dif_percent = dif*100 # получаем процент

dif_percent 


Или все то же самое можно записать в одну строку

In [ ]:
np.mean(np.abs(forecast.tail(31).reset_index()['yhat'] - test_0['y'])/test_0['y'])*100

Попробуем теперь сделать такой же прогноз, но отдельно по филиалам и сравним результаты.  
Для начала подготовим данные отдельно по филиалам, сначала данные для обучения:

1. отфильтруем для каждого филиала нужные строки
2. переименуем столбцы для работы с `Prophet`

Мы переименовывали столбцы, меняя значения атрибута `columns`, теперь используем метод `rename()` — с ним вы тоже будете часто сталкиваться в чужом коде. Параметр `axis = 1` указывает, что переименовываем колонки.

In [ ]:
train_1 = train.loc[train['filial'] == 1,['date','revenue']].rename({'date':'ds','revenue':'y'},axis=1) 
train_2 = train.loc[train['filial'] == 2,['date','revenue']].rename({'date':'ds','revenue':'y'},axis=1)
train_3 = train.loc[train['filial'] == 3,['date','revenue']].rename({'date':'ds','revenue':'y'},axis=1)

И то же самое проделаем для подготовки данных, чтобы проверить качества модели, только возьмем исходный тестовый датасет. В нем данные за декабрь. 

In [ ]:
test_1 = test.loc[test['filial'] == 1,['date','revenue']].rename({'date':'ds','revenue':'y'},axis=1)
test_2 = test.loc[test['filial'] == 2,['date','revenue']].rename({'date':'ds','revenue':'y'},axis=1)
test_3 = test.loc[test['filial'] == 3,['date','revenue']].rename({'date':'ds','revenue':'y'},axis=1)

Теперь строим три модели по разным данным. Для каждой модели заведем переменную, так как нам нужны три разные модели.

1. Заведем 3 разные модели для 3 филиалов с помощью объекта `Prophet()`.
2. Передадим для каждой модели обучающий датасет в метод `fit()`, чтобы подогнать модель под наши данные.

In [ ]:
m_1 = Prophet() # модель первого филиала
m_1.fit(train_1) # подгоняем модель под наши данные

# аналогичный код для двух других филиалов

m_2 = Prophet()
m_2.fit(train_2)

m_3 = Prophet()
m_3.fit(train_3)

Теперь нам предстоит построить прогноз на тот же период для трех моделей.

Применим к каждой модели метод `predict()`, передав ему датасет с «будущими» датами.



In [ ]:
forecast_1 = m_1.predict(future) # датасет с прогнозом для первого филиала
forecast_2 = m_2.predict(future) # датасет с прогнозом для второго филиала
forecast_3 = m_3.predict(future) # датасет с прогнозом для третьего филиала

Быстро визуализируем получившиеся данные

In [ ]:
m_1.plot(forecast_1); 

In [ ]:
m_2.plot(forecast_2);

In [ ]:
m_3.plot(forecast_3);

На графиках видно, что модель отдает слишком грубое предсказание. Попробуем скорректировать его с помощью более тонких настроек.

__Для начала посмотрим на ошибки отдельно для каждого филиала.__

Для этого для каждого филиала передадим в функцию `mean_absolute_error()` серию предсказанных значений и серию контрольных значений

In [ ]:
error_forecast_1 = mean_absolute_error(forecast_1['yhat'].tail(31), test_1['y'])
error_forecast_2 = mean_absolute_error(forecast_2['yhat'].tail(31), test_2['y'])
error_forecast_3 = mean_absolute_error(forecast_3['yhat'].tail(31), test_3['y'])

error_forecast_1, error_forecast_2, error_forecast_3

__Найдем процент ошибки для каждого филиала.__

Передадим в функцию `np.abs()` разницу между предсказанными данными и данными из тестовой выборки. Не забудем про `.reset_index()`.

Передадим в функцию `np.mean()` деление модулей разниц, которые получили только что, на данные тестовой выборки и умножим на 100, чтобы получить `%`.

In [ ]:
modul_dif_1 = np.abs(forecast_1.tail(31).reset_index()['yhat'] - test_1.reset_index()['y'])
dif_percent_1 = np.mean(modul_dif_1/test_1.reset_index()['y'])*100

modul_dif_2 = np.abs(forecast_2.tail(31).reset_index()['yhat'] - test_2.reset_index()['y'])
dif_percent_2 = np.mean(modul_dif_2/test_2.reset_index()['y'])*100

modul_dif_3 = np.abs(forecast_3.tail(31).reset_index()['yhat'] - test_3.reset_index()['y'])
dif_percent_3 = np.mean(modul_dif_3/test_3.reset_index()['y'])*100

dif_percent_1, dif_percent_2, dif_percent_3

В разных филиалах очень разный процент ошибки, давайте попробуем улучшить прогноз для первого и третьего филиалов.

## Настройка параметров модели

Мы запускали модель без параметров, давайте посмотрим, какие параметры модели можно задать. Рассмотрим основные:
- __`growth`__ — тренд 
    - `linear` — линейный, то есть данные растут или убывают равномерно
    
    - `logistic` — логистический сложносочиненный микс динамики данных
    
    
- __`holidays`__ — дата-фрейм с описанием праздников/выходных дней, чтобы они учитывались при построении прогноза


- __`seasonality`__ — сезонность

    - `weekly_seasonality` — недельная сезонность 

    - `daily_seasonality` — дневная сезонность 

    - `yearly_seasonality` — годовая сезонность 
    

- __`seasonality_mode`__ 
    - `multiplicative` — мультипликативная 
    - `additive` — аддитивная
    
Аддитивную сезонность имеет смысл использовать, если амплитуда колебаний сезонности из года в год не меняется. Если амплитуда колебаний сезонности из года в год меняется (т. е. размах уменьшается или увеличивается), то используем мультипликативную сезонность.

Вытащим тренд и обозначим, что у нас мультипликативная сезонность

In [ ]:
m_1 = Prophet(growth = 'linear', weekly_seasonality = True, seasonality_mode='multiplicative')
m_1.fit(train_1)

m_2 = Prophet(growth = 'linear', weekly_seasonality = True, seasonality_mode='multiplicative')
m_2.fit(train_2)

m_3 = Prophet(growth = 'linear', weekly_seasonality = True, seasonality_mode='multiplicative')
m_3.fit(train_3)

In [ ]:
forecast_1 = m_1.predict(future)
forecast_2 = m_2.predict(future)
forecast_3 = m_3.predict(future)

__Сравним ошибки__

В модели без параметров ошибки были: 
- 6270.750421944441
- 1627.752566953797
- 13210.181303922529

In [ ]:
error_forecast_1 = mean_absolute_error(forecast_1['yhat'].tail(31), test_1['y'])
error_forecast_2 = mean_absolute_error(forecast_2['yhat'].tail(31), test_2['y'])
error_forecast_3 = mean_absolute_error(forecast_3['yhat'].tail(31), test_3['y'])

error_forecast_1, error_forecast_2, error_forecast_3

__И посчитаем в процентах, так гораздо надежнее__ 

Напомню, что было:
- 6.7900069525078095
- 1.7779055372059944
- 26.902829147537332

In [ ]:
modul_dif_1 = np.abs(forecast_1.tail(31).reset_index()['yhat'] - test_1.reset_index()['y'])
dif_percent_1 = np.mean(modul_dif_1/test_1.reset_index()['y'])*100

modul_dif_2 = np.abs(forecast_2.tail(31).reset_index()['yhat'] - test_2.reset_index()['y'])
dif_percent_2 = np.mean(modul_dif_2/test_2.reset_index()['y'])*100

modul_dif_3 = np.abs(forecast_3.tail(31).reset_index()['yhat'] - test_3.reset_index()['y'])
dif_percent_3 = np.mean(modul_dif_3/test_3.reset_index()['y'])*100

dif_percent_1, dif_percent_2, dif_percent_3

In [ ]:
# хотя, датасайнтисты запишут скорее так:

np.mean(np.abs(forecast_1.tail(31).reset_index()['yhat'] - test_1.reset_index()['y'])/test_1.reset_index()['y'])*100, \
np.mean(np.abs(forecast_2.tail(31).reset_index()['yhat'] - test_2.reset_index()['y'])/test_2.reset_index()['y'])*100, \
np.mean(np.abs(forecast_3.tail(31).reset_index()['yhat'] - test_3.reset_index()['y'])/test_3.reset_index()['y'])*100

Теперь даже если мы сложим наши ожидаемые ошибки по трем филиалам, увидим, что общий прогноз стал гораздо точнее.

In [ ]:
f1 = forecast_1.tail(31).reset_index()['yhat']
f2 = forecast_2.tail(31).reset_index()['yhat']
f3 = forecast_3.tail(31).reset_index()['yhat']

total_forecast = f1 + f2 + f3

mean_absolute_error(total_forecast, test_0['y'])

Теперь, когда модель выбрана, надо перестроить модель по всему набору данных и сделать прогноз на январь:

In [ ]:
# готовим данные с мая по декабрь

data_revenue_1 = data_revenue.loc[data_revenue['filial'] == 1].drop('filial', axis=1).rename({'date':'ds','revenue':'y'},axis=1)
data_revenue_2 = data_revenue.loc[data_revenue['filial'] == 2].drop('filial', axis=1).rename({'date':'ds','revenue':'y'},axis=1)
data_revenue_3 = data_revenue.loc[data_revenue['filial'] == 3].drop('filial', axis=1).rename({'date':'ds','revenue':'y'},axis=1)

# обучаем модель

m_1 = Prophet(growth = 'linear', weekly_seasonality = True, seasonality_mode='multiplicative')
m_1.fit(data_revenue_1)

m_2 = Prophet(growth = 'linear', weekly_seasonality = True, seasonality_mode='multiplicative')
m_2.fit(data_revenue_2)

m_3 = Prophet(growth = 'linear', weekly_seasonality = True, seasonality_mode='multiplicative')
m_3.fit(data_revenue_3)

# создаем дата-фрейм на январь

future = m_1.make_future_dataframe(periods=31) # говорим профету сделать дата-фрейм на 31 день

# наполняем январь данными для каждого филиала

forecast_1 = m_1.predict(future)
forecast_2 = m_2.predict(future)
forecast_3 = m_3.predict(future)



### Немного читерства

Теперь мы хотим объединить данные по филиалам. Тут нам мог бы пригодиться `merge()`, но вспомним, что мы можем объединять только 2 таблицы, то есть придется делать две операции объединения. А если в таблице одинаковые колонки, то мы можем просто сложить дата-фреймы.

Чтобы воспользоваться этой суперспособностью дата-фреймов, нам нужно, чтобы дата оказалась в индексе, ведь даты в `pandas` не складываются.

In [ ]:
# складываем выручку на каждый день по всем 3 филиалам

f1 = forecast_1.tail(31)[['ds','yhat']].set_index('ds')
f2 = forecast_2.tail(31)[['ds','yhat']].set_index('ds')
f3 = forecast_3.tail(31)[['ds','yhat']].set_index('ds')

total_forecast = (f1 + f2 + f3)

In [ ]:
total_forecast.head()

Возьмем только нужные столбцы и переименуем их для заказчика.

In [ ]:
total_forecast.reset_index(inplace=True)
total_forecast[['ds', 'yhat']]
total_forecast.columns = ['date', 'revenue']

Выгрузим данные в таблицу.

In [ ]:
total_forecast.head()

In [ ]:
total_forecast.to_csv('forecast_jan2021.csv')

<body>
  <div
    style="
      padding-top: 298px;
      padding-bottom: 298px;
      background-color: black;
      border-radius: 24px;
    "
  >
    <div style="text-align: center">
      <div
        style="
          font-size: 72px;
          line-height: 80px;
          font-family: Arial;
          color: white;
        "
      >
        Спасибо за внимание
      </div>
    </div>
  </div>
</body>
